In [1]:
import numpy as np
import pandas as pd
import seaborn as sns 
import torch as t 
import torch.nn as nn 
import torch.optim as optim
from torch.utils.data import TensorDataset,DataLoader
from torchtext.data.utils import get_tokenizer
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("/Users/bhara-zstch1566/RNN/Project/Sentiment analysis/Dataset/sentiment_data.csv")

In [3]:
df

,Unnamed: 0,Comment,Sentiment
0,0,lets forget apple pay required brand new iphon...,1
1,1,nz retailers don’t even contactless credit car...,0
2,2,forever acknowledge channel help lessons ideas...,2
3,3,whenever go place doesn’t take apple pay doesn...,0
4,4,apple pay convenient secure easy use used kore...,2
...,...,...,...
241140,241921,crores paid neerav modi recovered congress lea...,0
241141,241922,dear rss terrorist payal gawar modi killing pl...,0
241142,241923,cover interaction forum left,1
241143,241924,big project came india modi dream project happ...,1


In [4]:
df = df.drop('Unnamed: 0',axis=1)

In [5]:
df

,Comment,Sentiment
0,lets forget apple pay required brand new iphon...,1
1,nz retailers don’t even contactless credit car...,0
2,forever acknowledge channel help lessons ideas...,2
3,whenever go place doesn’t take apple pay doesn...,0
4,apple pay convenient secure easy use used kore...,2
...,...,...
241140,crores paid neerav modi recovered congress lea...,0
241141,dear rss terrorist payal gawar modi killing pl...,0
241142,cover interaction forum left,1
241143,big project came india modi dream project happ...,1


In [6]:
Y = df['Sentiment'].to_numpy()


In [7]:
X = df['Comment']

In [8]:
X = X.fillna(" ")

In [10]:
np.unique(Y)

array([0, 1, 2])

In [11]:
mx = 0
for i in X:
    mx = max(mx,len(i))
mx

5080

In [12]:
tokenizer = get_tokenizer('basic_english')


In [13]:
for i in range(len(X)):
    X[i] = tokenizer(X[i])

In [14]:
X

0         [lets, forget, apple, pay, required, brand, ne...
1         [nz, retailers, don’t, even, contactless, cred...
2         [forever, acknowledge, channel, help, lessons,...
3         [whenever, go, place, doesn’t, take, apple, pa...
4         [apple, pay, convenient, secure, easy, use, us...
                                ...                        
241140    [crores, paid, neerav, modi, recovered, congre...
241141    [dear, rss, terrorist, payal, gawar, modi, kil...
241142                    [cover, interaction, forum, left]
241143    [big, project, came, india, modi, dream, proje...
241144    [ever, listen, like, gurukul, discipline, main...
Name: Comment, Length: 241145, dtype: object

In [15]:
text = []
for i in X:
    for j in i:
        text.append(j)


In [16]:
len(text)

3133195

In [17]:
idx = {i:j for j,i in enumerate(sorted(set(text)))}

In [18]:
idx

{'aa': 0,
 'aaa': 1,
 'aaaa': 2,
 'aaaaa': 3,
 'aaaaaaaaaaa': 4,
 'aaaaaaaaaahhhhhhhh': 5,
 'aaaaaaaaaamazing': 6,
 'aaaaaaaafternoon': 7,
 'aaaaaaaahhhhhhhh': 8,
 'aaaaaah': 9,
 'aaaaaahhhhhhhh': 10,
 'aaaaaalcohol': 11,
 'aaaaaawwwesome': 12,
 'aaaaagg': 13,
 'aaaaahhhh': 14,
 'aaaaall': 15,
 'aaaaand': 16,
 'aaaaandnothin': 17,
 'aaaaaoouoouoouu': 18,
 'aaaaarrrrggghhh': 19,
 'aaaaaw': 20,
 'aaaaawwwwwww': 21,
 'aaaagggessss': 22,
 'aaaagh': 23,
 'aaaah': 24,
 'aaaahh': 25,
 'aaaahhhhhhhhhh': 26,
 'aaaaiieee': 27,
 'aaaallll': 28,
 'aaaand': 29,
 'aaaargh': 30,
 'aaaau': 31,
 'aaaaw': 32,
 'aaaawww': 33,
 'aaadhar': 34,
 'aaah': 35,
 'aaahaha': 36,
 'aaahh': 37,
 'aaahhh': 38,
 'aaahhhjust': 39,
 'aaak': 40,
 'aaall': 41,
 'aaamoo': 42,
 'aaand': 43,
 'aaao': 44,
 'aaap': 45,
 'aaapke': 46,
 'aaargh': 47,
 'aaarrrgggghhh': 48,
 'aaarrrgh': 49,
 'aaas': 50,
 'aaauuuggghhh': 51,
 'aaaw': 52,
 'aaaww': 53,
 'aaawww': 54,
 'aaayen': 55,
 'aab': 56,
 'aabar': 57,
 'aabe': 58,
 'aabhas': 

In [19]:
len(idx)

158985

In [20]:
numeric = {idx[i]:i for i in idx}

In [21]:
numeric

{0: 'aa',
 1: 'aaa',
 2: 'aaaa',
 3: 'aaaaa',
 4: 'aaaaaaaaaaa',
 5: 'aaaaaaaaaahhhhhhhh',
 6: 'aaaaaaaaaamazing',
 7: 'aaaaaaaafternoon',
 8: 'aaaaaaaahhhhhhhh',
 9: 'aaaaaah',
 10: 'aaaaaahhhhhhhh',
 11: 'aaaaaalcohol',
 12: 'aaaaaawwwesome',
 13: 'aaaaagg',
 14: 'aaaaahhhh',
 15: 'aaaaall',
 16: 'aaaaand',
 17: 'aaaaandnothin',
 18: 'aaaaaoouoouoouu',
 19: 'aaaaarrrrggghhh',
 20: 'aaaaaw',
 21: 'aaaaawwwwwww',
 22: 'aaaagggessss',
 23: 'aaaagh',
 24: 'aaaah',
 25: 'aaaahh',
 26: 'aaaahhhhhhhhhh',
 27: 'aaaaiieee',
 28: 'aaaallll',
 29: 'aaaand',
 30: 'aaaargh',
 31: 'aaaau',
 32: 'aaaaw',
 33: 'aaaawww',
 34: 'aaadhar',
 35: 'aaah',
 36: 'aaahaha',
 37: 'aaahh',
 38: 'aaahhh',
 39: 'aaahhhjust',
 40: 'aaak',
 41: 'aaall',
 42: 'aaamoo',
 43: 'aaand',
 44: 'aaao',
 45: 'aaap',
 46: 'aaapke',
 47: 'aaargh',
 48: 'aaarrrgggghhh',
 49: 'aaarrrgh',
 50: 'aaas',
 51: 'aaauuuggghhh',
 52: 'aaaw',
 53: 'aaaww',
 54: 'aaawww',
 55: 'aaayen',
 56: 'aab',
 57: 'aabar',
 58: 'aabe',
 59: 'aabha

In [22]:
for i in X:
    for j in range(len(i)):
        i[j] = idx[i[j]]

In [23]:
X

0         [75944, 48462, 7023, 100066, 113082, 17834, 92...
1         [94562, 113624, 38082, 43448, 28133, 30037, 20...
2         [48421, 1543, 22276, 57997, 75897, 61569, 4446...
3         [146871, 52497, 102435, 37674, 131397, 7023, 1...
4         [7023, 100066, 28438, 119616, 40005, 141817, 1...
                                ...                        
241140    [30514, 98052, 91503, 85431, 111408, 27472, 75...
241141    [32626, 115315, 133145, 100069, 51112, 85431, ...
241142                         [29628, 65108, 48667, 75599]
241143    [15128, 106730, 19962, 63145, 85431, 38751, 10...
241144    [43499, 76899, 76534, 55114, 36657, 79829, 434...
Name: Comment, Length: 241145, dtype: object

In [24]:
mx = 0 
for i in X:
    mx = max(mx,len(i))

In [25]:
mx

742

In [26]:
X = [t.tensor(i) for i in X]

X = pad_sequence(X,True,padding_value=0)

In [27]:
X.shape

torch.Size([241145, 742])

In [28]:
Y =t.from_numpy(Y)

In [29]:
Y 

tensor([1, 0, 2,  ..., 1, 1, 2])

In [30]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.7,shuffle=True)
print(type(x_train), type(y_train))


<class 'torch.Tensor'> <class 'torch.Tensor'>


In [31]:
train_loader = TensorDataset(x_train,y_train)
train_loader = DataLoader(train_loader,64,True)

In [32]:
test_loader = TensorDataset(x_test,y_test)
test_loader = DataLoader(test_loader,64,True)

In [33]:
class SentimentModel(nn.Module):
    def __init__(self, voc, emb, hid):
        super().__init__()
        self.embadding = nn.Embedding(voc,emb)
        self.lstm = nn.LSTM(emb,hid,batch_first=True)
        self.fc = nn.Linear(hid,3)

    def forward(self,x):
        x = self.embadding(x)

        _,(h,_) = self.lstm(x)
        h = h[-1]
        return (self.fc(h))


In [36]:
model= SentimentModel(len(idx),128,128)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)
device = 'mps' if t.mps.is_available() else 'cpu'
model = model.to(device)
print(device)

mps


In [37]:
for i in range(5):
    running_loss = 0.0
    for x,y in train_loader:
        x,y = x.to(device),y.to(device)
        y_pred = model(x)
        loss = criterion(y_pred,y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
    print(f"epoch : {i+1} and loss : {running_loss/len(train_loader):4f}")


epoch : 1 and loss : 1.069578
epoch : 2 and loss : 1.068855
epoch : 3 and loss : 1.068726
epoch : 4 and loss : 1.068744
epoch : 5 and loss : 1.068690


In [ ]:
t.mps.empty_cache()
